#################################################

** Created by Prof. Dr. Marcelo Oliveira on 08/2021. 

** Disciplina de Fundametos de IA aplicados ao Diagnóstico de Doenças. 

** Copyright (c)2021 Instituto de Computação ** 

** Universidade Federal de Alagoas - UFAL. All rights reserved.**

################################################# 

References:

### Requerimentos: pydicom, scikit-image, kornia, fastai 

In [1]:
#pip install git+https://github.com/fastai/fastai.git
!pip install fastai --upgrade

     |████████████████████████████████| 186 kB 5.6 MB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [2]:
!pip show fastai

Name: fastai
Version: 2.5.2
Summary: fastai simplifies training fast and accurate neural nets using modern best practices
Home-page: https://github.com/fastai/fastai/tree/master/
Author: Jeremy Howard, Sylvain Gugger, and contributors
Author-email: info@fast.ai
License: Apache Software License 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: torch, requests, pillow, torchvision, fastprogress, fastcore, scikit-learn, pyyaml, spacy, pip, fastdownload, scipy, packaging, pandas, matplotlib
Required-by: 


In [3]:
pip install pydicom

     |████████████████████████████████| 2.0 MB 5.3 MB/s 


In [4]:
pip install kornia

     |████████████████████████████████| 303 kB 5.1 MB/s 


In [5]:
import os
import numpy as np
import shutil
import random

import fastai

from fastai.vision.all import *
from fastcore.transform import Pipeline

from fastai.data.block import CategoryBlock, DataBlock
from fastai.data.core import Datasets
from fastai.data.external import untar_data, URLs
from fastai.data.transforms import Categorize, GrandparentSplitter, IntToFloatTensor, Normalize, RandomSplitter, ToTensor, parent_label

from fastai.torch_core import to_device

from fastai.vision.augment import aug_transforms, Resize, RandomResizedCrop, FlipItem
from fastai.vision.data import ImageBlock, PILImage, get_image_files, imagenet_stats
from fastai.medical.imaging import *

import pydicom,kornia,skimage

In [6]:
!touch "/content/drive/My Drive/ping.txt"
source=Path("/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/")
files = os.listdir(source)
print(files)

['COVID', 'normal']


In [7]:
#dataSetPath = get_dicom_files(root_dir)
#dataSetPath

In [8]:
#train = source/'train'
train = source
train_files = get_dicom_files(train)
train_files

(#3402) [Path('/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/COVID/4e0367ed8dd0.dcm'),Path('/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/COVID/4e078c23b8cd.dcm'),Path('/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/COVID/4e30d63e57de.dcm'),Path('/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/COVID/4e8a9f4212c7.dcm'),Path('/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/COVID/4e8f69e4ffa4.dcm'),Path('/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/COVID/4ec4f6830669.dcm'),Path('/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/COVID/4efcdede6982.dcm'),Path('/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/COVID/4f0c18a7a451.dcm'),Path('/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/COVID/4f11b62bab24.dcm'),Path('/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/COVID/4f257ce09b76.dcm')...]

In [9]:
#patient1 = train_files[7]
#dimg = pydicom.dcmread(patient1)
#dimg

In [10]:
def show_one_patient(file):
    """ function to view patient image and choosen tags within the head of the DICOM"""
    pat = pydicom.dcmread(file)
    print(f'patient Name: {pat.PatientName}')
    print(f'Patient ID: {pat.PatientID}')
    #print(f'Patient age: {pat.PatientAge}')
    print(f'Patient Sex: {pat.PatientSex}')
    print(f'Body part: {pat.BodyPartExamined}')
  

In [12]:
def label_func(fname):
    return print(fname.name[0])#"COVID" if fname.name[0].isupper() else "normal"

In [13]:
blocks = (ImageBlock(cls=PILDicom), CategoryBlock)

In [14]:
data_block = DataBlock(blocks=blocks,
                       #get_items = get_image_files,
                       #get_x=get_dicom_files,
                       get_x=get_image_files,
                       get_y     = parent_label,#label_func,
                       splitter  = RandomSplitter(valid_pct=0.2, seed=42),
                       #splitter  = GrandparentSplitter(),
                       item_tfms=Resize(128)
                      # batch_tfms=aug_transforms(size=224)
                       )

#dls = data_block.dataloaders(dataSetPath)
dls = data_block.dataloaders(train_files)
#dls = DataLoaders.from_dsets(dataSetPath)
#dls = dls.cuda()
#data_block.summary(train_files)
dls.vocab

['COVID', 'normal']

In [15]:
data_block.summary(train_files)

Setting-up type transforms pipelines
Found 3402 items
2 datasets of sizes 2722,680
Setting up Pipeline: get_image_files -> PILDicom.create
Setting up Pipeline: parent_label -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: get_image_files -> PILDicom.create
    starting from
      /content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/normal/42833e3ae880.dcm
    applying get_image_files gives
      []
    applying PILDicom.create gives
      []
  Pipeline: parent_label -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/normal/42833e3ae880.dcm
    applying parent_label gives
      normal
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(1)

Final sample: ([], TensorCategory(1))


Found 3402 items
2 datasets of sizes 2722,680
Setting up Pipeline: get_image_files -> PILDicom.create
Setting up

In [16]:
learn = cnn_learner(dls, 
                    resnet18, 
                    metrics=[accuracy, error_rate])                

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [17]:
learn.fine_tune(4)

epoch,train_loss,valid_loss,accuracy,error_rate,time


TypeError: ignored

In [ ]:


#covidNormal.train[900]
#dls = covidNormal.dataloaders(dataSetPath)

In [ ]:
teste = (f'{dataSetPath}/train')

In [ ]:
a = lambda x:teste/f"{x[2]}"
print(a)

In [ ]:
#trn_path = get_dicom_files(f'{dataSetPath}/train')
#val_path  = get_dicom_files(f'{dataSetPath}/test')

In [ ]:
#dcm =  train_images[0]
#dcm.dcmread()

In [ ]:
#trn_path = Path("/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/")
#dl = ImageDataLoaders.from_folder(trn_path, train='train', bs=8, valid_pct=.1, seed=42)
#dl.train.show_batch()

In [ ]:
#dls = ImageDataLoaders.from_folder(path=items, 
#                                   train='train', 
#                                   valid='val', 
#                                   shuffle=True, 
#                                   item_tfms=Resize(448))